In [ ]:
import pandas as pd 
import numpy as np 
# Load AIS data 
ais_data = pd.read_csv("ais_dataset.csv")  # Replace with actual file path 
# Load each emission factor file 
propulsion_factors = pd.read_csv("propulsion_engine_emission_factors.csv") 
boiler_factors = pd.read_csv("boiler_engine_emission_factors.csv") 
auxiliary_factors = pd.read_csv("auxiliary_engine_emission_factors.csv") 
# Add a column to identify engine type 
propulsion_factors["engine_type"] = "propulsion" 
boiler_factors["engine_type"] = "boiler" 
auxiliary_factors["engine_type"] = "auxiliary" 
# Combine all emission factor files into one DataFrame 
emission_factors = pd.concat([propulsion_factors, boiler_factors, auxiliary_factors], ignore_index=True) 
# Display the combined data 
print(emission_factors.head()) 
# Step 1: Merge AIS data with emission factors based on vessel type or ID 
merged_data = ais_data.merge(emission_factors, on="vessel_type")  # Adjust 'vessel_type' as per your dataset 
# Step 2: Calculate total time in anchorage (hours) 
merged_data["anchorage_time_hours"] = (pd.to_datetime(merged_data["departure_time"]) -  
                                       pd.to_datetime(merged_data["arrival_time"])).dt.total_seconds() / 3600 
# Step 3: Calculate emissions for each vessel 
merged_data["emission_before_jit"] = (merged_data["fuel_consumption_rate"] *  
                                      merged_data["anchorage_time_hours"] *  
                                      merged_data["emission_factor"]) 
# Step 4: Aggregate emissions 
anc_before_jit = merged_data["emission_before_jit"].sum() 
print(f"Total CO2 emissions before JIT: {anc_before_jit} tonnes")


                                  fuel  fuel_category sulfur engine_type  \
0  HFO (outside ECA and prior to 2020)              1  2.70%  propulsion   
1  HFO (outside ECA and prior to 2020)              1  2.70%  propulsion   
2  HFO (outside ECA and prior to 2020)              1  2.70%  propulsion   
3  HFO (outside ECA and prior to 2020)              1  2.70%  propulsion   
4  HFO (outside ECA and prior to 2020)              1  2.70%  propulsion   

   emission_tier model_years   nox  pm10  pm2_5    voc     sox   n2_O   ch4  \
0            0.0    Pre-2000  18.1  1.42   1.31  0.632  10.293  0.031  0.01   
1            1.0   2000-2010  17.0  1.42   1.31  0.632  10.293  0.031  0.01   
2            2.0   post 2010  15.3  1.42   1.31  0.632  10.293  0.031  0.01   
3            0.0    Pre-2000  14.0  1.43   1.32  0.530  11.244  0.030  0.01   
4            1.0   2000-2010  13.0  1.43   1.32  0.530  11.244  0.030  0.01   

    sfoc  n_ox  bsfc engine_size  s_ox  n2_o  
0  195.0   NaN   NaN 

KeyError: 'vessel_type'

In [28]:
import pandas as pd

# Load datasets
ais_data = pd.read_csv('ais_dataset.csv')
aux_engine_factors = pd.read_csv('auxiliary_engine_emission_factors.csv')
boiler_engine_factors = pd.read_csv('boiler_engine_emission_factors.csv')

# Filter engine factors for matching fuel_category
aux_engine_factors = aux_engine_factors[aux_engine_factors['fuel_category'] == 2]
boiler_engine_factors = boiler_engine_factors[boiler_engine_factors['fuel_category'] == 2]

# Step 1: Filter for anchorage events
anchorage_data = ais_data[(ais_data['speed'] < 1) & (ais_data['anchorage'] == 1)].copy()

# Step 2: Convert 'timestamp' to datetime and drop invalid rows
anchorage_data['timestamp'] = pd.to_datetime(anchorage_data['timestamp'], errors='coerce')
anchorage_data.dropna(subset=['timestamp'], inplace=True)

# Filter for July
july_data = anchorage_data[anchorage_data['timestamp'].dt.month == 7]

# Step 3: Merge datasets on 'fuel_category'
print("Before merge:", july_data.shape)
aux_merged = pd.merge(july_data, aux_engine_factors, on='fuel_category', how='left')
print("After auxiliary merge:", aux_merged.shape)
boiler_merged = pd.merge(aux_merged, boiler_engine_factors, on='fuel_category', how='left')
print("After boiler merge:", boiler_merged.shape)

# Step 4: Calculate emissions
boiler_merged['activity_duration_hours'] = (
    boiler_merged['timestamp'].diff().dt.total_seconds() / 3600
)
boiler_merged['aux_emissions'] = (
    boiler_merged['ael'] * boiler_merged['activity_duration_hours'] * boiler_merged['bsfc_x']
)
boiler_merged['boiler_emissions'] = (
    boiler_merged['abl'] * boiler_merged['activity_duration_hours'] * boiler_merged['bsfc_y']
)
boiler_merged['total_emissions'] = (
    boiler_merged['aux_emissions'] + boiler_merged['boiler_emissions']
)

# Step 5: Aggregate emissions for July
july_emissions = boiler_merged.groupby('imo')['total_emissions'].sum().reset_index()
july_emissions.columns = ['imo', 'anc_before_jit']
july_emissions['anc_before_jit'] /= 1000  # Convert grams to tonnes

# Save the results
print(july_emissions)
july_emissions.to_csv('anc_before_jit.csv', index=False)


Before merge: (0, 30)
After auxiliary merge: (0, 43)
After boiler merge: (0, 53)
Empty DataFrame
Columns: [imo, anc_before_jit]
Index: []


In [10]:
print(ais_data.columns)


Index(['imo', 'mmsi', 'vessel_name', 'date_of_build', 'vessel_type', 'group',
       'timestamp', 'lon', 'lat', 'nav_stat', 'speed', 'course', 'heading',
       'fuel_category', 'main_engine_fuel_type', 'aux_engine_fuel_type',
       'engine_type', 'berth', 'port_name', 'anchorage', 'terminal',
       'maneuvering_zone', 'p', 'vref', 'sfc_me', 'sfc_ae', 'sfc_ab', 'ael',
       'abl', 'distance'],
      dtype='object')


In [18]:
print(ais_data['timestamp'].head())
print(ais_data['timestamp'].dtype)







0    2024-07-28T00:23:32.000Z
1    2024-07-28T00:25:02.000Z
2    2024-07-28T00:30:01.000Z
3    2024-07-28T00:32:51.000Z
4    2024-07-28T00:35:02.000Z
Name: timestamp, dtype: object
object


In [20]:
print("AIS Data Columns:", ais_data.columns)
print("Auxiliary Engine Factors Columns:", aux_engine_factors.columns)
print("Boiler Engine Factors Columns:", boiler_engine_factors.columns)


AIS Data Columns: Index(['imo', 'mmsi', 'vessel_name', 'date_of_build', 'vessel_type', 'group',
       'timestamp', 'lon', 'lat', 'nav_stat', 'speed', 'course', 'heading',
       'fuel_category', 'main_engine_fuel_type', 'aux_engine_fuel_type',
       'engine_type', 'berth', 'port_name', 'anchorage', 'terminal',
       'maneuvering_zone', 'p', 'vref', 'sfc_me', 'sfc_ae', 'sfc_ab', 'ael',
       'abl', 'distance'],
      dtype='object')
Auxiliary Engine Factors Columns: Index(['fuel', 'fuel_category', 'sulfur', 'engine_size', 'emission_tier',
       'model_years', 'nox', 'pm10', 'pm2_5', 'voc', 's_ox', 'n2_o', 'ch4',
       'bsfc'],
      dtype='object')
Boiler Engine Factors Columns: Index(['fuel', 'fuel_category', 'sulfur', 'n_ox', 'pm10', 'pm2_5', 'voc',
       'sox', 'n2_O', 'ch4', 'bsfc'],
      dtype='object')


In [23]:
print(ais_data['fuel_category'].unique())
print(aux_engine_factors['fuel_category'].unique())
print(boiler_engine_factors['fuel_category'].unique())


[2]
[1 2 3]
[1 2 3]


In [24]:
print("Before merge:", july_data.shape)
aux_merged = pd.merge(july_data, aux_engine_factors, on='fuel_category', how='left')
print("After auxiliary merge:", aux_merged.shape)
boiler_merged = pd.merge(aux_merged, boiler_engine_factors, on='fuel_category', how='left')
print("After boiler merge:", boiler_merged.shape)


Before merge: (0, 30)
After auxiliary merge: (0, 43)
After boiler merge: (0, 53)


In [31]:
print("Fuel categories in July data:", july_data['fuel_category'].unique())


Fuel categories in July data: []


In [32]:
anchorage_data = ais_data[ais_data['anchorage'].notnull()].copy()


In [35]:
july_data = anchorage_data[anchorage_data['timestamp'].dt.month == 7]


In [ ]:
anchorage_data = ais_data[ais_data['anchorage'].notnull()].copy()


In [37]:
# Step 3: Merge datasets on 'fuel_category'
aux_engine_factors = pd.read_csv('auxiliary_engine_emission_factors.csv')
boiler_engine_factors = pd.read_csv('boiler_engine_emission_factors.csv')

# Filter engine factors for matching fuel_category
aux_engine_factors = aux_engine_factors[aux_engine_factors['fuel_category'] == 2]
boiler_engine_factors = boiler_engine_factors[boiler_engine_factors['fuel_category'] == 2]

# Merge auxiliary engine factors
aux_merged = pd.merge(july_data, aux_engine_factors, on='fuel_category', how='left')
print("After auxiliary merge:", aux_merged.shape)

# Merge boiler engine factors
boiler_merged = pd.merge(aux_merged, boiler_engine_factors, on='fuel_category', how='left')
print("After boiler merge:", boiler_merged.shape)

# Step 4: Calculate emissions
# Calculate duration between timestamps in hours
boiler_merged['activity_duration_hours'] = (
    boiler_merged['timestamp'].diff().dt.total_seconds() / 3600
)

# Auxiliary engine emissions: E = L_A * A * EF
boiler_merged['aux_emissions'] = (
    boiler_merged['ael'] * boiler_merged['activity_duration_hours'] * boiler_merged['bsfc_x']
)

# Boiler emissions: E = L_B * A * EF
boiler_merged['boiler_emissions'] = (
    boiler_merged['abl'] * boiler_merged['activity_duration_hours'] * boiler_merged['bsfc_y']
)

# Total emissions at anchorage
boiler_merged['total_emissions'] = (
    boiler_merged['aux_emissions'] + boiler_merged['boiler_emissions']
)

# Step 5: Aggregate emissions for July
july_emissions = boiler_merged.groupby('imo')['total_emissions'].sum().reset_index()
july_emissions.columns = ['imo', 'anc_before_jit']
july_emissions['anc_before_jit'] /= 1000  # Convert grams to tonnes

# Save the results to a CSV file
print("Final emissions data:")
print(july_emissions.head())
july_emissions.to_csv('anc_before_jit.csv', index=False)


After auxiliary merge: (1471404, 43)
After boiler merge: (1471404, 53)
Final emissions data:
       imo  anc_before_jit
0  1013315   -17009.030660
1  1014838    -6457.638222
2  1015820    24836.123125
3  1017775     6518.825733
4  1018547    99971.276731
